In [ ]:
from tensorflow.keras.layers import *
import tensorflow as tf
import numpy as np
import gc
from tensorflow import keras
from keras import layers
from keras.models import Model

In [ ]:
! pip install allensdk

In [3]:
from pathlib import Path
from allensdk.core.brain_observatory_cache import BrainObservatoryCache
boc = BrainObservatoryCache(cache= True,manifest_file='../brain_data/brain_observatory_manifest.json')
nwb = boc.get_ophys_experiment_data(ophys_experiment_id=500964514)

2023-10-28 20:07:25,347 allensdk.api.api.retrieve_file_over_http INFO     Downloading URL: http://api.brain-map.org/api/v2/well_known_file_download/514408643


In [ ]:
from data_loading import load_data
_,_,out = load_data(experiment_id=500964514)